In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils.FFT import spectrogram, stft

c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:

proxy = 'http://proxy-k.ecl.ntt.co.jp:8080'

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [3]:
TEST = False

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
L.seed_everything(random_seed)

Seed set to 1


1

## FFT Data

In [4]:
data_key = ["LAcc_x", "LAcc_y", "LAcc_z", "Gyr_x", "Gyr_y", "Gyr_z"]

data_path_dict = dict(zip(data_key, 
    [os.path.join("dataset", "Torso", "LAcc_x.txt"),
    os.path.join("dataset", "Torso", "LAcc_y.txt"),
    os.path.join("dataset", "Torso", "LAcc_z.txt"),
    os.path.join("dataset", "Torso", "Gyr_x.txt"),
    os.path.join("dataset", "Torso", "Gyr_y.txt"),
    os.path.join("dataset", "Torso", "Gyr_z.txt"),]))

In [5]:
data_dict = {}

for data_name in data_key:
    one_data = np.loadtxt(data_path_dict[data_name])
    data_dict[data_name] = stft(one_data, verbose=True)[1]

data_dict.keys()


data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)
data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)
data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)
data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)
data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)
data1d.shape (98036000,)
freqs.shape (257,)
specs.shape (257, 196072)


dict_keys(['LAcc_x', 'LAcc_y', 'LAcc_z', 'Gyr_x', 'Gyr_y', 'Gyr_z'])

In [6]:
stack = False

In [7]:
## STACK

if stack:
    STACK_FREQ_NUMBER = 8

    one_data = data_dict[data_name]
    data_array_list = []

    for freq_step in range(STACK_FREQ_NUMBER):
        start_idx = freq_step
        end_idx = - STACK_FREQ_NUMBER + freq_step + 1
        if end_idx == 0:
            end_idx = None

        data_array = one_data[:, start_idx: end_idx]
        print("data_array.shape", data_array.shape)
        data_array_list.append(data_array)

    one_data_stacked = np.stack(data_array_list)
    print("one_data_stacked.shape", one_data_stacked.shape)

    data_stacked_list = []

    for data_name in data_key:
        one_data = data_dict[data_name]
        data_array_list = []

        for freq_step in range(STACK_FREQ_NUMBER):
            start_idx = freq_step
            end_idx = - STACK_FREQ_NUMBER + freq_step + 1
            if end_idx == 0:
                end_idx = None

            data_array = one_data[:, start_idx: end_idx]
            data_array_list.append(data_array)

        one_data_stacked = np.stack(data_array_list)
        data_stacked_list.append(one_data_stacked)

    data_stacked = np.stack(data_stacked_list)
    print("data_stacked.shape", data_stacked.shape)

    data_stacked = data_stacked.transpose(3, 0, 1, 2)
    print("data_stacked.shape", data_stacked.shape)

    np.save(os.path.join("dataset", "processed_data", "torso_fft.npy") ,data_stacked)
    

In [8]:
## NO STACK
if not stack:
    data_stacked_list = []

    for data_name in data_key:
        print(data_dict[data_name].shape)
        data_stacked_list.append(data_dict[data_name])
        
    data_stacked = np.stack(data_stacked_list)
    print("data_stacked.shape", data_stacked.shape)

    data_stacked = data_stacked.transpose(2, 0, 1)
    print("data_stacked.shape", data_stacked.shape)
    
    np.save(os.path.join("dataset", "processed_data", "torso_fft.npy") ,data_stacked)
    

(257, 196072)
(257, 196072)
(257, 196072)
(257, 196072)
(257, 196072)
(257, 196072)
data_stacked.shape (6, 257, 196072)
data_stacked.shape (196072, 6, 257)


## Label

In [9]:
label_path = os.path.join("dataset", "Torso", "Label.txt")
label = np.loadtxt(label_path)
label.shape

(196072, 500)

In [10]:
if stack:
    data_label = label[: - STACK_FREQ_NUMBER + 1, :]
    print(data_label.shape)
    np.save(os.path.join("dataset", "processed_data", "torso_label.npy"), data_label)

In [11]:
if not stack:
    np.save(os.path.join("dataset", "processed_data", "torso_label.npy"), label)